In [83]:
from dotenv import load_dotenv
from os import getenv
from pprint import pprint
import time
import pandas as pd
import duckdb

from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks import OpenAICallbackHandler

# SQL agent
from langchain_community.utilities import SQLDatabase
from langchain.agents import create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
import sqlalchemy

load_dotenv("../.env")

True

### List of models to be tested

In [31]:
models = {
    "llama_free": "meta-llama/llama-3.3-8b-instruct:free",
    "deepseek_free": "deepseek/deepseek-r1-0528-qwen3-8b:free",
    "gemma_free": "google/gemma-3n-e4b-it:free",
    "gemini": "google/gemini-2.5-flash-lite",
    "llama_lunaris": "sao10k/l3-lunaris-8b",
    "mistral": "mistralai/ministral-8b"
}

In [32]:
# Test model availability
def test_model_availability(model: str) -> bool:
    client = OpenAI(
        base_url=getenv("OPENROUTER_API_URL"),
        api_key=getenv("OPENROUTER_API_KEY"),
    )
    try:
        client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "test"}],
            max_tokens=1
        )
        return True
    except Exception as e:
        error_msg = str(e).lower()
        if "region" in error_msg or "country" in error_msg or "blocked" in error_msg:
            print(f"Model {model} is region-blocked.")
            return False  # Region-blocked
        else:
            pprint(e)  # Print other errors for debugging
            return False

# models = {
#     model_name: model_id for model_name, model_id in models.items()
#     if test_model_availability(model_id)
# }

In [33]:
# Timer function to measure execution time

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Execution time: {round(end_time - start_time, 2)} seconds")
        return result
    return wrapper

### Create simple LangChain example

In [50]:
# Create first langchain agent to prompt LLM
template = """
Question: {question}
Answer: Let's think step by step.
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["question"]
)

llm = ChatOpenAI(
    api_key=getenv("OPENROUTER_API_KEY"),
    base_url=getenv("OPENROUTER_API_URL"),
    model_name=models["llama_free"],
    max_completion_tokens=256
)

llm_chain = prompt | llm | StrOutputParser()

@timer
def langchain_prompt(question: str) -> str:
    return llm_chain.invoke(question)
langchain_prompt("What is the president of the USA?")

Execution time: 4.14 seconds


'To find the answer to who the President of the USA is, we can follow a logical process that involves checking the most recent information available and verifying through reliable sources. Here\'s a step-by-step approach:\n\n1. **Stay Updated with Current News**: The first step is to stay updated with the latest news. Check reputable news websites, such as CNN, BBC, or the official website of the White House, for the latest political news.\n\n2. **Check Official Websites**: Visit the official website of the White House (whitehouse.gov) or the official website of the U.S. Government (usa.gov) for the most current information on the President of the United States.\n\n3. **Use Search Engines**: Search engines like Google can be very helpful. Simply typing "President of the USA" or "Who is the current President of the United States?" can give you the latest information.\n\n4. **Verify through Multiple Sources**: To ensure the accuracy of the information, verify it through multiple sources.

### Connect to DB to fetch data

In [67]:
# Connect to local duckdb to fetch data

conn = duckdb.connect(
    database="../data/housing_crawler_read_only.duckdb",
    read_only=True
)

@timer
def query_housing_data(query: str) -> pd.DataFrame:
    df = conn.execute(query).df()
    # Remove id fields to minimize data size
    id_fields = [col for col in df.columns if "id" in col.lower()]
    df = df.drop(columns=id_fields)
    return df

def df_to_markdown(df: pd.DataFrame) -> str:
    return df.to_markdown(index=False)

df = query_housing_data("SELECT * FROM estate_info LIMIT 2;")
df_markdown = df_to_markdown(df)
print(df_markdown)


Execution time: 0.01 seconds
| name       | region_name   | subregion_name   | district_name             | sm_district_name          |   location_lat |   location_lon | first_op_date       |   sell_count |   rent_count |   yearly_tx_count |   yearly_total_tx_amount |   yearly_net_ft_price |   yearly_net_ft_price_chg |   recent_ft_price |   recent_net_ft_price |   recent_tx_count |   recent_net_ft_price_chg |   recent_pre_net_ft_price |   recent_ft_price_chg |   recent_pre_ft_price |   recent_total_tx_amount | developer_name   | parent_estate_name   |
|:-----------|:--------------|:-----------------|:--------------------------|:--------------------------|---------------:|---------------:|:--------------------|-------------:|-------------:|------------------:|-------------------------:|----------------------:|--------------------------:|------------------:|----------------------:|------------------:|--------------------------:|--------------------------:|----------------------:|---------

### Use Markdown template for SQL generation using LLM

In [68]:
# Use markdown table in prompt

template_with_data = """
Here is an extract of the table called estate_info from Hong Kong Housing data from DuckDB:
{table_data}
Question: {question}
Create SQL query to answer the question based on the data above.
The SQL query will be executed on the same data.
The respond should only contains the SQL query in plain text, not in code block.
Do not include explanations or additional text.
"""

prompt_with_data = PromptTemplate(
    template=template_with_data,
    input_variables=["table_data", "question"]
)
llm_chain_with_data = prompt_with_data | llm | StrOutputParser()

token_count = OpenAICallbackHandler()
response = llm_chain_with_data.with_config(callbacks=[token_count]).invoke(input={
    "table_data": df_markdown, 
    "question": "Which estate has the highest transaction volume in Kowloon?"
    })
print(response)
print(token_count.total_tokens)  # Check token usage


SELECT name FROM estate_info WHERE region_name = 'Kowloon' ORDER BY yearly_tx_count DESC LIMIT 1;
600


### Create SQL agent directly instead of prompt template

In [ ]:
llm = ChatOpenAI(
    api_key=getenv("OPENROUTER_API_KEY"),
    base_url=getenv("OPENROUTER_API_URL"),
    model_name=models["llama_lunaris"],
    max_completion_tokens=256
)

db = SQLDatabase.from_uri(
    "duckdb:///../data/housing_crawler_read_only.duckdb"
)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,  # Add verbose for debugging
    handle_parsing_errors=True
)

@timer
def langchain_sql_agent(question: str) -> str:
    return agent.run(question)

langchain_sql_agent("Which estate has the highest transaction volume in Kowloon?")

/Users/tc/Documents/Github/HK_Housing_Agent_Eval/env/lib/python3.12/site-packages/duckdb_engine/__init__.py:184: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: estate_infoNow that I know there's a table named 'estate_info', it relates to the question. I think I should get the schema to see what information it contains to construct a suitable query.

Action: sql_db_schema
Action Input: estate_info
CREATE TABLE estate_info (
	id VARCHAR, 
	name VARCHAR, 
	region_id VARCHAR, 
	region_name VARCHAR, 
	subregion_id VARCHAR, 
	subregion_name VARCHAR, 
	district_id VARCHAR, 
	district_name VARCHAR, 
	sm_district_id VARCHAR, 
	sm_district_name VARCHAR, 
	location_lat FLOAT, 
	location_lon FLOAT, 
	first_op_date DATE, 
	sell_count INTEGER, 
	rent_count INTEGER, 
	yearly_tx_count INTEGER, 
	yearly_total_tx_amount FLOAT, 
	yearly_net_ft_price FLOAT, 
	yearly_net_ft_price_chg FLOAT, 
	recent_ft_price FLOAT, 
	recent_net_ft_price FLOAT, 
	recent_tx_count INTEGER, 
	recent_net_ft_price_chg FLOAT, 
	recent_pre_net_ft_price FLOAT, 
	recent_ft_price_chg FLOAT, 
	recent_pre_ft

'Lohas Park has the highest transaction volume in Kowloon, with 46 recent transactions as of the provided data.'